<center>
<font color='#1DD2AF'>
<h1>Crecimiento de Fracturas y su Adaptación Geométrica en Espacios Hexaedrizados</h1>
<h3>Julian Ricardo Salazar Duarte</h3>
<h3> Profesor: Marco Paluszny Kluczynsky</h3>
<h3>Universidad Nacional de Colombia</h3>
<h3>2024-1</h3>
</font>
</center>

### 0. configuración del entorno

In [1]:
%pip install -r requirements.txt

In [2]:
import numpy as np
from matplotlib import use
import plotly.graph_objects as go
from fracture.fracture import Fracture

### 1. Hexaedrizacion de la fractura

In [3]:
# Carga los datos de la fractura desde un archivo de texto.

# FRAC0003_nrIter4.txt
# FRAC0006_nrIter27.txt
# FRAC0019_nrIter27.txt

filename = 'FRAC0003_nrIter4.txt'
dataset = []
with open(filename, 'r') as file:
    for line in file:
        values = [float(value) for value in line.split()]
        dataset.append(values)

In [4]:
f = Fracture(dataset, 1e-3)
prismas = None
hexaedros = None
x_axis = f.x_axis
y_axis = f.y_axis
max_value = f.max_value
isdegenerate = f.isdegenerate
mp = f.mp
vertex = f.vertex
points = f.points

In [5]:
def update_plot(fig, title):

    fig.update_layout(
        title = title,
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z',
            xaxis=dict(range=[mp[0]- max_value-2, mp[0] + max_value+2]),
            yaxis=dict(range=[mp[1]- max_value-2, mp[1] + max_value+2]),
            zaxis=dict(range=[mp[2]- max_value-2, mp[2] + max_value+2]),
        )
    )

Los prismas se generan a partir de los triángulos, donde cada prisma tiene 6 vértices, los 3 puntodes del triángulo de la fractura y los 3 puntos del triangulo correspondiente de la caja contenedora.

In [6]:
prismas = []
for i in f.tri:
    prisma = np.array([f.M[i[0]], f.M[i[1]], f.M[i[2]], f.inter[i[0]], f.inter[i[1]], f.inter[i[2]]])
    prismas.append(np.array(prisma))
prismas = np.array(prismas)

In [7]:
def plot_prismas(prismas, fig, isdegenerate):
    for i, prisma in enumerate(prismas):
        if i in isdegenerate:
            c = 'red'
        else:
            c = 'blue'

        # Triángulo 1
        triangle1 = np.array([prisma[0], prisma[1], prisma[2], prisma[0]])
        fig.add_trace(go.Scatter3d(
            x=triangle1[:, 0],
            y=triangle1[:, 1],
            z=triangle1[:, 2],
            mode='lines',
            line=dict(color=c)
        ))

        # Triángulo 2
        triangle2 = np.array([prisma[3], prisma[4], prisma[5], prisma[3]])
        fig.add_trace(go.Scatter3d(
            x=triangle2[:, 0],
            y=triangle2[:, 1],
            z=triangle2[:, 2],
            mode='lines',
            line=dict(color=c, dash='dash')
        ))

        # Caras laterales
        for j in range(3):
            fig.add_trace(go.Scatter3d(
                x=[prisma[j][0], prisma[j + 3][0]],
                y=[prisma[j][1], prisma[j + 3][1]],
                z=[prisma[j][2], prisma[j + 3][2]],
                mode='lines',
                line=dict(color=c, dash='dash')
            ))

In [8]:
fig = go.Figure()
f.build_box(fig)
plot_prismas(prismas, fig, isdegenerate)
update_plot(fig, 'Prismas')
fig.show()

 Los hexaedros se generan a partir de pares de triángulos consecutivos. Cada hexaedro tiene 8 vértices: los 4 de los dos triángulos y los 4 puntos donde las normales de esos vértices intersectan la caja.

In [9]:
hexaedros = []
for i in range(0, f.tri.shape[0], 2):

    hexaedro = np.array([f.M[f.tri[i][0]], f.M[f.tri[i][1]], f.M[f.tri[i][2]], f.M[f.tri[i+1][2]],
            f.inter[f.tri[i][0]], f.inter[f.tri[i][1]], f.inter[f.tri[i][2]], f.inter[f.tri[i+1][2]]])
    
    hexaedros.append(np.array(hexaedro))

hexaedros = np.array(hexaedros)

In [10]:
def plot_hexaedro(fig, hexaedros, isdegenerate):

    for i, prisma in enumerate(prismas):
        if i in isdegenerate:
            c = 'red'
        else:
            c = 'blue'

    for i, hexaedro in enumerate(hexaedros):
        # color = f'rgb({np.random.rand(1)[0]*255}, {np.random.rand(1)[0]*255}, {np.random.rand(1)[0]*255})'

        # Cara 1
        quad = np.array([hexaedro[0], hexaedro[1], hexaedro[2], hexaedro[3], hexaedro[0]])
        fig.add_trace(go.Scatter3d(
            x=quad[:, 0],
            y=quad[:, 1],
            z=quad[:, 2],
            mode='lines',
            line=dict(color=c)
        ))

        # Cara 2
        quad = np.array([hexaedro[4], hexaedro[5], hexaedro[6], hexaedro[7], hexaedro[4]])
        fig.add_trace(go.Scatter3d(
            x=quad[:, 0],
            y=quad[:, 1],
            z=quad[:, 2],
            mode='lines',
            line=dict(color=c, dash='dash')
        ))

        # Caras laterales
        for j in range(4):
            fig.add_trace(go.Scatter3d(
                x=[hexaedro[j][0], hexaedro[j + 4][0]],
                y=[hexaedro[j][1], hexaedro[j + 4][1]],
                z=[hexaedro[j][2], hexaedro[j + 4][2]],
                mode='lines',
                line=dict(color=c, dash='dash')
            ))

In [11]:
fig = go.Figure()
f.build_box(fig)
plot_hexaedro(fig, hexaedros, isdegenerate)
update_plot(fig, 'hexaedros')
fig.show()

### 2. construir la retícula (hexaedrizar el espacio)

Se divide el espacio alrededor de la fractura en cubos unitarios para luego evaluar si estos cubos intersectan con los hexaedros que representan la geometría de la fractura

In [12]:
subcubos = []

min_vals = np.floor(np.min(vertex, axis=0)).astype(int)
max_vals = np.ceil(np.max(vertex, axis=0)).astype(int)

x = np.arange(min_vals[0], max_vals[0] + 1)
y = np.arange(min_vals[1], max_vals[1] + 1)
z = np.arange(min_vals[2], max_vals[2] + 1)

# Generar cubos unitarios en la dirección original
for xi in np.arange(min_vals[0], max_vals[0]):
    for yi in np.arange(min_vals[1], max_vals[1]):
        for zi in np.arange(min_vals[2], max_vals[2]):
            cubo = np.array([
                [xi, yi, zi],
                [xi + 1, yi, zi],
                [xi + 1, yi + 1, zi],
                [xi, yi + 1, zi],
                [xi, yi, zi + 1],
                [xi + 1, yi, zi + 1],
                [xi + 1, yi + 1, zi + 1],
                [xi, yi + 1, zi + 1]
            ])
            subcubos.append(cubo)

subcubos = np.array(subcubos)

In [13]:
def plot_reticula(fig, subcubos):   # definir lineas de cada hexaedro
    lines = [
        [0, 1], [0, 3], [0, 4],
        [1, 2], [1, 5], [2, 3],
        [2, 6], [3, 7], [4, 5],
        [4, 7], [5, 6], [6, 7]
    ]


    # Draw the subcubes
    for subcubo in subcubos:
        vertices = np.array(subcubo)
        color = np.random.rand(3,)
        for line in lines:
            fig.add_trace(go.Scatter3d(
                x=[vertices[line[0], 0], vertices[line[1], 0]],
                y=[vertices[line[0], 1], vertices[line[1], 1]],
                z=[vertices[line[0], 2], vertices[line[1], 2]],
                mode='lines',
                line=dict(color=f'rgb({color[0]*255},{color[1]*255},{color[2]*255})', width=2)
                )
            )
    #fig.write_html('regilla.html')


In [14]:
fig = go.Figure()
f.build_box(fig)
plot_hexaedro(fig, hexaedros, isdegenerate)
plot_reticula(fig, subcubos)
update_plot(fig, 'Reticula')
fig.show()

### 3. Intersección de subcubos y hexaedros

In [15]:
import trimesh
import numpy as np
from trimesh.collision import CollisionManager

def check_collision(vertices_hex1, vertices_hex2):

    # Definir los vértices de dos hexaedros


    # Definir las caras de un hexaedro (6 caras con 4 vértices cada una)
    faces = np.array([
        [0, 1, 2], [0, 2, 3],  # Cara inferior
        [4, 5, 6], [4, 6, 7],  # Cara superior
        [0, 1, 5], [0, 5, 4],  # Cara frontal
        [1, 2, 6], [1, 6, 5],  # Cara derecha
        [2, 3, 7], [2, 7, 6],  # Cara trasera
        [3, 0, 4], [3, 4, 7]   # Cara izquierda
    ])

    # Crear mallas para los dos hexaedros
    hex1 = trimesh.Trimesh(vertices=vertices_hex1, faces=faces)
    hex2 = trimesh.Trimesh(vertices=vertices_hex2, faces=faces)

    # Crear un gestor de colisiones
    collision_manager = CollisionManager()

    # Añadir las mallas al gestor de colisiones
    collision_manager.add_object('hex1', hex1)
    collision_manager.add_object('hex2', hex2)

    # Comprobar si los dos hexaedros se intersectan
    return collision_manager.in_collision_internal()


In [16]:
intersections = None
def collisions():
    global intersections, hexaedros, subcubos

    intersections = [[] for _ in range(len(hexaedros))]

    for i, hex in enumerate(hexaedros):
        hex = np.array([
            hex[3], hex[7], hex[4], hex[0],  # Base inferior
            hex[2], hex[6], hex[5], hex[1]   # Base superior
        ])
        for j, cubo in enumerate(subcubos):
            if check_collision(hex, cubo):
                intersections[i].append(j)

In [17]:
collisions()

In [18]:
def plot_intersections(fig, hexaedros, subcubos, intersections, idx):
    
    plot_hexaedro(fig, np.array([hexaedros[idx]]), [])
    cubos = subcubos[intersections[idx]]
    plot_reticula(fig, cubos)

            

In [19]:
fig = go.Figure()
plot_intersections(fig, hexaedros, subcubos, intersections, 30)
fig.show()

La idea final del proceso es estudiar las intersecciones para que la hexaedrización externa (los cubos unitarios en el espacio) esté coherente con la hexaedrización interna (los hexaedros de la fractura). Es decir, asegurarnos de que los cubos que intersectan con la fractura están ajustados correctamente a los hexaedros de la fractura, sin grandes cambios entre iteraciones.